Extract Data from jobvision..

In [10]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from urllib.parse import urlparse, parse_qs

In [15]:
options = Options()
options.add_argument("--headless=new") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)
base_url="https://jobvision.ir/jobs?page="

jobs_list = []
now = datetime.now()

driver.get(base_url + "1")

pages_btn = WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "pagination"))
    )

btn_list = pages_btn.find_elements(By.XPATH, "./*")
last_page = btn_list[-1]

last_page.click()

current_url = driver.current_url
parsed_url = urlparse(current_url)
query_params = parse_qs(parsed_url.query)
pages = int(query_params.get('page')[0])


for page in range(1, pages + 1):
    url = base_url + str(page)
    driver.get(url)

    jobs = WebDriverWait(driver, 15).until(
        EC.visibility_of_element_located((By.TAG_NAME, "job-card-list"))
    )

    jobs_element_list = jobs.find_elements(By.TAG_NAME, "job-card")

    for job in jobs_element_list:

        # Get information about job postings from post

        title = job.find_element(By.CLASS_NAME, "job-card-title")

        parent = title.find_element(By.XPATH, "..")
        childs_list = parent.find_elements(By.XPATH, "./*")

        company = childs_list[1]

        job_info_element = childs_list[2]

        if len(childs_list) == 5:
            facility = childs_list[3].text
        else:
            facility = ""
        shipping_time = childs_list[-1]

        job_info_list = job_info_element.find_elements(By.TAG_NAME, "span")

        if len(job_info_list) == 3:
            region, _, salary = job_info_list
            salary = salary.text
        else:
            region = job_info_list[0]
            salary = ""

        extractec_num = "".join(filter(lambda x: x.isnumeric(), shipping_time.text))
        if extractec_num != "":
            release_time = now - timedelta(days=int(extractec_num))
        else:
            release_time = now - timedelta(days=0)

        # Go to any of the job position

        title.click()

        get_job_section = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.TAG_NAME, "app-job-specification"))
        )

        job_info_list = get_job_section.find_elements(By.CLASS_NAME, "text-muted")
        working_hours, type_of_cooperation = job_info_list[:2]
        try:
            requirement_list = get_job_section.find_elements(By.TAG_NAME, "app-tag")
            requirement_software_list = [i.text for i in requirement_list]
        except:
            requirement_software_list = []

        jobs_list.append(
            {
                "shippingDate": release_time.date(),
                "title": title.text,
                "company": company.text,
                "salary": salary,
                "province": region.text,
                "workingHours": working_hours.text,
                "typeOfCooperation": type_of_cooperation.text,
                "facility": facility,
                "tagList": requirement_software_list,
            }
        )


driver.quit()

df = pd.DataFrame(jobs_list)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6EADDEFA5+77893]
	GetHandleVerifier [0x00007FF6EADDF000+77984]
	(No symbol) [0x00007FF6EABA91BA]
	(No symbol) [0x00007FF6EABFF16D]
	(No symbol) [0x00007FF6EABFF41C]
	(No symbol) [0x00007FF6EAC52237]
	(No symbol) [0x00007FF6EAC2716F]
	(No symbol) [0x00007FF6EAC4F07F]
	(No symbol) [0x00007FF6EAC26F03]
	(No symbol) [0x00007FF6EABF0328]
	(No symbol) [0x00007FF6EABF1093]
	GetHandleVerifier [0x00007FF6EB097B6D+2931725]
	GetHandleVerifier [0x00007FF6EB092132+2908626]
	GetHandleVerifier [0x00007FF6EB0B00F3+3031443]
	GetHandleVerifier [0x00007FF6EADF91EA+184970]
	GetHandleVerifier [0x00007FF6EAE0086F+215311]
	GetHandleVerifier [0x00007FF6EADE6EC4+110436]
	GetHandleVerifier [0x00007FF6EADE7072+110866]
	GetHandleVerifier [0x00007FF6EADCD479+5401]
	BaseThreadInitThunk [0x00007FFB3F1A6FD4+20]
	RtlUserThreadStart [0x00007FFB40F7CEC1+33]


In [6]:
df_copy = df

df_copy.to_csv('job_data_optimized.csv', index=False)